# HW1

In [54]:
import numpy as np
import pandas as pd
import os, glob, tqdm, time
from tqdm import tqdm, trange
import multiprocessing
from multiprocessing import Pool
from sklearn import preprocessing

import networkx as nx
from torch_geometric.data import Data, InMemoryDataset, DataLoader
import torch
import torch.nn as nn
from torch.nn import Linear, Parameter, Sequential
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import degree, add_self_loops, convert
import torch.nn.functional as F
from torch_geometric.utils.convert import to_networkx, from_networkx
from torch.utils.data import Dataset, DataLoader, TensorDataset


import matplotlib.pyplot as plt

/home/rita/miniconda3/envs/jupyterlab/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [55]:
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


## Read Data

In [56]:
os.chdir('/home/rita/111/111-2MLG/HW1')
os.getcwd()

'/home/rita/111/111-2MLG/HW1'

In [57]:
youtube_edges = np.array(pd.read_table('./hw1_data/youtube/com-youtube.txt', sep = ' '))
youtube_scores = np.array(pd.read_table('./hw1_data/youtube/com-youtube_score.txt', header = None))
print(type(youtube_edges))
print(type(youtube_scores))
print(youtube_edges.shape)
print(youtube_scores.shape)
print(youtube_edges[:2])
print(youtube_scores[:2])

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(2987623, 2)
(1134890, 2)
[[0 2]
 [0 3]]
[['0:' 29320492.911274]
 ['1:' 445191851.627834]]


In [58]:
def set_init(graph) :
    row, col = graph.edge_index
    row = row.unsqueeze(0)
    col = col.unsqueeze(0)
    t = torch.hstack([row, col])
    t = t.squeeze()

    deg = degree(t).float().unsqueeze(1)
    print
    extend = torch.ones(graph.num_nodes, 2)
    init = torch.hstack([deg, extend])
    graph.x = init
    
    return graph

In [18]:
# 生成圖
youtube = Data(edge_index = torch.from_numpy(youtube_edges.T), num_nodes = youtube_edges.max() + 1)
# print(youtube.x.shape)
youtube = set_init(youtube)
print(type(youtube))
print(len(youtube))
print(youtube.x.shape)
print(youtube.x[0])
print(f'Number of nodes: {youtube.num_nodes}')
print(f'Number of edges: {youtube.num_edges}')

<class 'torch_geometric.data.data.Data'>
3
torch.Size([1134890, 3])
tensor([28.,  1.,  1.])
Number of nodes: 1134890
Number of edges: 2987623


In [19]:
path = '/home/rita/111/111-2MLG/HW1/hw1_data/Synthetic/5000'
os.chdir(path)
files = glob.glob('*')
file_name = [x[:-4] for x in files]

total_data = {}
for i in range(len(files)) :
    total_data[file_name[i]] = np.array(pd.read_table(files[i], header = None))

print(len(total_data))
print(type(total_data))
print(type(total_data[file_name[0]]))
print(type(total_data[file_name[-1]]))
print(total_data[file_name[0]][:2])
print(total_data[file_name[-1]][:2])

60
<class 'dict'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[0.         0.04482959]
 [1.         0.11496664]]
[[0 4]
 [0 5]]


In [20]:
# 生成圖
total_graph = {}
for i in range(len(files)//2):
    edge = total_data[str(i)]
    temp = Data(edge_index =  torch.from_numpy(edge.T), num_nodes = edge.max() + 1)
    total_graph[i] = set_init(temp)
print(len(total_graph))
print(len(total_graph[0]))
print(type(total_graph))
print(type(total_graph[0]))
print(type(total_graph[29]))
print(total_graph[0])
print(total_graph[29])
print(total_graph[0].x)

30
3
<class 'dict'>
<class 'torch_geometric.data.data.Data'>
<class 'torch_geometric.data.data.Data'>
Data(edge_index=[2, 19982], num_nodes=5000, x=[5000, 3])
Data(edge_index=[2, 19983], num_nodes=5000, x=[5000, 3])
tensor([[239.,   1.,   1.],
        [178.,   1.,   1.],
        [149.,   1.,   1.],
        ...,
        [  4.,   1.,   1.],
        [  4.,   1.,   1.],
        [  4.,   1.,   1.]])


In [22]:
path = '/home/rita/111/111-2MLG/HW1/hw1_data/Synthetic/5000'
os.chdir(path)
files = glob.glob('*score.*')
file_name = [x[:-10] for x in files]

total_score = {}
for i in range(len(file_name)) :
    total_score[file_name[i]] = np.array(pd.read_table(files[i]))
print(total_score['0'])

[[1.00000000e+00 5.39707966e-02]
 [2.00000000e+00 4.43436579e-02]
 [3.00000000e+00 2.23256726e-02]
 ...
 [4.99700000e+03 1.95420222e-05]
 [4.99800000e+03 6.05783053e-05]
 [4.99900000e+03 1.09075567e-04]]


In [26]:
os.chdir('/home/rita/111/111-2MLG/HW1')
os.getcwd()

'/home/rita/111/111-2MLG/HW1'

In [27]:
# GCNconv
# 修改成 Algorithm 裡面的樣子
class GCNConv_self(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add')  # "Add" aggregation (Step 5).

    def forward(self, x, edge_index):
        row, col = edge_index
        row = row.unsqueeze(0)
        col = col.unsqueeze(0)
        t = torch.hstack([row, col])
        t = t.squeeze()
        
        deg = degree(t, x.size(0), dtype=x.dtype) + 1
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        out = self.propagate(edge_index, x=x, norm=norm)

        return out

    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

In [28]:
# DrBC
# ENCODER DECODER
class DrBC(nn.Module) :
    def __init__(self, encoder_layers = 5, emb_dim = 128) :
        super(DrBC, self).__init__()
        # encoder
        self.encoder_layers = encoder_layers
        self.emb_dim = emb_dim
        # self.n_nodes = n_nodes
        self.line2 = nn.Sequential(
            nn.Linear(3, self.emb_dim),  # w0 (5000 * p)
            nn.ReLU()
        )
        self.line5 = GCNConv_self(self.emb_dim, self.emb_dim)
        # w1, u1, w2, u2, w3, u3  (p * p)
        self.line6 = nn.GRUCell(input_size = self.emb_dim, hidden_size = self.emb_dim, bias = False) # (p * p)
        
        # decoder
        self.decoder = nn.Sequential(
            nn.Linear(self.emb_dim, 32),  # w4
            nn.ReLU(), 
            nn.Linear(32, 1),  # w5
            # nn.Sigmoid()
        )
        
    def forward(self, graph) :
        # encoder
        hv_0 = graph.x
        hv = self.line2(hv_0)
        hv = F.normalize(hv) # hv / |hv|2
        
        total_hv = hv.unsqueeze(0).clone().detach()
        # total_hv = torch.tensor(hv.unsqueeze(0))
        for i in range(2, self.encoder_layers + 1) :
            hnv = self.line5(hv, graph.edge_index)
            hv = self.line6(hnv, hv)
            hv = F.normalize(hv)
            total_hv = torch.vstack((total_hv, hv.unsqueeze(0))) # (3, 500, 64)
        
        z_v, _ = torch.max(total_hv, dim = 0)
        
        # decoder
        out = self.decoder(z_v)

        return out
model = DrBC().to(device)
print(model)

DrBC(
  (line2): Sequential(
    (0): Linear(in_features=3, out_features=128, bias=True)
    (1): ReLU()
  )
  (line5): GCNConv_self()
  (line6): GRUCell(128, 128, bias=False)
  (decoder): Sequential(
    (0): Linear(in_features=128, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=1, bias=True)
  )
)


In [29]:
# batch
# new
def training_loop(model, n_epochs, optimizer) : # , loss_fn, train_loader
    nodes = 200
    deg = 3
    p = 0.2
    num_g = 16
    # loss_fn = nn.BCEWithLogitsLoss(weight = torch.tensor(16000), reduction = 'sum')
    loss_fn = nn.BCEWithLogitsLoss()
    t = tqdm(range(1, n_epochs + 1), total = n_epochs)
    for epoch in t :   
        start = 0
        for i in range(num_g) :
            # Draw network G from distribution D (like the power-law model)
            g = nx.powerlaw_cluster_graph(nodes, deg, p)
            # [dv, 1, 1]
            degrees = list(dict(g.degree()).values())
            degree_tensor = torch.tensor(degrees).float().unsqueeze(1)
            extend = torch.ones(nodes, 2)
            init = torch.hstack([degree_tensor, extend])
            edge_index = torch.tensor(list(g.edges())) + start
            
            # Calculate each node’s exact BC value bv, ∀v ∈ V
            # 1-n
            exact_bc = nx.betweenness_centrality(g)
            exact_bc = torch.tensor(list(exact_bc.values()))
            
            # 隨機取排列組合
            input_tensor = torch.tensor(range(start, start + nodes))
            combinations_tensor = torch.combinations(input_tensor, r = 2)
            num_combinations = combinations_tensor.shape[0]
            indices = torch.randperm(num_combinations)[:5 * nodes]
            node_pair = combinations_tensor[indices]
            
            if (i == 0) : 
                total_init = init
                total_exact_bc = exact_bc
                total_node_pair = node_pair
                total_edge_index = edge_index
            else : 
                total_init = torch.vstack([total_init, init])
                total_exact_bc = torch.cat([total_exact_bc, exact_bc])
                total_node_pair = torch.vstack([total_node_pair, node_pair])
                total_edge_index = torch.vstack([total_edge_index, edge_index])
                
            start += nodes
            
        print(total_init.shape)   
        print(total_exact_bc.shape)  
        print(total_node_pair.shape)  
        print(total_edge_index.shape)  
        
        idx1 = total_node_pair[::, 0]
        idx2 = total_node_pair[::, 1]
        pyg_g = Data(edge_index = total_edge_index.T, num_nodes = total_edge_index.max() + 1)
        pyg_g.x = total_init
        pred_bc = model(pyg_g.to(device = device))
        # pred_bc = pred_bc.cpu()
        
        pred_a = pred_bc[idx1]
        pred_b = pred_bc[idx2]
        yij = pred_a - pred_b
        exact_a = total_exact_bc[idx1]
        exact_b = total_exact_bc[idx2]
        bij = exact_a - exact_b
        
        bij = bij.clone().detach().requires_grad_(True).to(device)
        yij = yij.clone().detach().requires_grad_(True)
        yij = yij.squeeze() # (n, )
        
        print(yij[:5])
        print(bij[:5])
        yij = yij * 10000
        bij = bij * 10000
        loss = loss_fn(yij, torch.sigmoid(bij))
        print(loss.item())
        # Update Θ = (ΘEN C, ΘDEC ) with Adam by minimizing Eq. (15)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
                
        t.set_description(f'Epoch[{epoch} / {n_epochs}]')
        t.set_postfix(loss = loss.item())


In [30]:
# parameters
learning_rate = 1e-4
embedding_dimension = 128
mini_batch_size = 16
average_node_sampling_times = 5
maximum_episodes = 10 ** 4
layer_iterations = 5
model = DrBC(encoder_layers = layer_iterations, emb_dim = embedding_dimension).to(device = device)
loss_fn = nn.BCEWithLogitsLoss(reduction = 'sum')
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay = 1e-5)
n_epochs = 20


training_loop(
    model = model, 
    n_epochs = n_epochs, 
    optimizer = optimizer, 
    # loss_fn = loss_fn, 
    
)
torch.save(model.state_dict(), './model/DrBC.pt')

Epoch[1 / 20]:   5%|▌         | 1/20 [00:01<00:30,  1.62s/it, loss=1.79]

torch.Size([3200, 3])
torch.Size([3200])
torch.Size([16000, 2])
torch.Size([9441, 2])
tensor([0.0107, 0.0064, 0.0056, 0.0029, 0.0104], device='cuda:0',
       grad_fn=<SliceBackward0>)
tensor([0.0223, 0.0377, 0.0067, 0.0036, 0.0719], device='cuda:0',
       grad_fn=<SliceBackward0>)
1.785771369934082


Epoch[2 / 20]:  10%|█         | 2/20 [00:03<00:29,  1.63s/it, loss=1.81]

torch.Size([3200, 3])
torch.Size([3200])
torch.Size([16000, 2])
torch.Size([9441, 2])
tensor([ 0.0042, -0.0020,  0.0043,  0.0075,  0.0031], device='cuda:0',
       grad_fn=<SliceBackward0>)
tensor([ 0.0053, -0.0024,  0.0029,  0.0078,  0.0017], device='cuda:0',
       grad_fn=<SliceBackward0>)
1.8139452934265137


Epoch[3 / 20]:  15%|█▌        | 3/20 [00:04<00:27,  1.64s/it, loss=1.78]

...

torch.Size([3200, 3])
torch.Size([3200])
torch.Size([16000, 2])
torch.Size([9446, 2])
tensor([0.0007, 0.0059, 0.0025, 0.0019, 0.0095], device='cuda:0',
       grad_fn=<SliceBackward0>)
tensor([-2.9630e-04,  5.2473e-03,  9.8777e-04, -6.5618e-05,  2.6223e-01],
       device='cuda:0', grad_fn=<SliceBackward0>)
1.8907876014709473


Epoch[19 / 20]:  95%|█████████▌| 19/20 [00:31<00:01,  1.63s/it, loss=1.66]

torch.Size([3200, 3])
torch.Size([3200])
torch.Size([16000, 2])
torch.Size([9428, 2])
tensor([0.0084, 0.0117, 0.0004, 0.0083, 0.0101], device='cuda:0',
       grad_fn=<SliceBackward0>)
tensor([ 0.1195,  0.0307, -0.0003,  0.1271,  0.0065], device='cuda:0',
       grad_fn=<SliceBackward0>)
1.6611322164535522


Epoch[20 / 20]: 100%|██████████| 20/20 [00:32<00:00,  1.63s/it, loss=1.91]

torch.Size([3200, 3])
torch.Size([3200])
torch.Size([16000, 2])
torch.Size([9437, 2])
tensor([-0.0028,  0.0014,  0.0035,  0.0003, -0.0005], device='cuda:0',
       grad_fn=<SliceBackward0>)
tensor([ 1.5531e-03,  3.7803e-01,  1.1181e-03,  1.0481e-04, -2.0205e-03],
       device='cuda:0', grad_fn=<SliceBackward0>)
1.9147696495056152


In [34]:
# accuracy
def top_n_accuracy(model, graph, exact_bc, n = 1) :
    num = int(np.ceil(graph.num_nodes * n / 100))
    exact = np.argsort(exact_bc[::, 1])[-num:]
    pred_bc = model(graph.to(device = device))
    pred = torch.argsort(pred_bc, dim = 0)[-num:].cpu()
    acc = len(np.intersect1d(exact, pred)) / num

    return exact, pred, acc
model = DrBC(encoder_layers = layer_iterations, emb_dim = embedding_dimension).to(device = device)
model.load_state_dict(torch.load('./model/DrBC.pt'))
exact, pred, acc = top_n_accuracy(model = model, graph = youtube, exact_bc = youtube_scores, n = 1)
print(acc)
print(exact.shape)
print(exact[:5])
print(pred.shape)
print(pred[:5])

0.46171468851881226
(11349,)
[184506  26813 358116   4056 159406]
torch.Size([11349, 1])
tensor([[ 26683],
        [115095],
        [199328],
        [ 29703],
        [ 13909]])


# Represent Table (Not Yet)

In [ ]:
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.") #可以根據輸出結果知道是否有可用的GPUa

In [ ]:
# parameters
learning_rate = 1e-4
embedding_dimension = 128
mini_batch_size = 16
average_node_sampling_times = 5
maximum_episodes = 10 ** 4
layer_iterations = 5
model = DrBC(encoder_layers = layer_iterations, emb_dim = embedding_dimension).to(device = device)
loss_fn = nn.BCEWithLogitsLoss(reduction = 'sum')
# loss_fn = nn.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # weight_decay = 1e-5
n_epochs = 5
model = DrBC(encoder_layers = layer_iterations, emb_dim = embedding_dimension).to(device = device)
model.load_state_dict(torch.load('./model/DrBC.pt'))
model.to(device)

In [ ]:
import scipy.stats as stats
data = youtube
score = youtube_scores[::, 1]

pred = model(data.to(device))
exact_bc = score

tau_dist, _ = stats.kendalltau(pred.cpu().detach().numpy(), exact_bc)
print('Kendall tau distance : ', '.4f'%tau_dist)